In [105]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [106]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [78]:
g = np.array([[ 1, 2, 3, 4, 5], [3, 6, 9, 12, 15], [5, 4, 3, 2, 1 ] ])
print(g)
print(g[:, (1, 4)])

[[ 1  2  3  4  5]
 [ 3  6  9 12 15]
 [ 5  4  3  2  1]]
[[ 2  5]
 [ 6 15]
 [ 4  1]]


In [42]:
a = np.array([[ 1, 2, 3, 4, 5], [3, 6, 9, 12, 15], [5, 4, 3, 2, 1 ] ])
print(a)
missings = np.array([])
print(missings.shape)
N = a.shape[1]
d = a.shape[0]
for i in range(N):
    print(i)
    col = a[:,i]
    if (col[col==3].shape[0] != 0):
        miss = col.copy()
        miss[miss != 3] = 0
        miss[miss == 3] = 1
        col[col == 3] = 0
        miss = miss.reshape(-1, 1)
        if (missings.shape[0] == 0):
            missings = miss
        else:
             missings = np.append(missings, miss, axis = 1)
        print(missings)

a = np.append(a, missings, axis = 1)
print(a)

[[ 1  2  3  4  5]
 [ 3  6  9 12 15]
 [ 5  4  3  2  1]]
(0,)
0
[[0]
 [1]
 [0]]
1
2
[[0 1]
 [1 0]
 [0 1]]
3
4
[[ 1  2  0  4  5  0  1]
 [ 0  6  9 12 15  1  0]
 [ 5  4  0  2  1  0  1]]


In [107]:
print(tX[tX == -999].shape)
missing_vectors = np.array([])
n_par = tX.shape[1]
for i in range(n_par):
    col = tX[:,i]
    if (col[col==-999].shape[0] != 0):
        miss = col.copy()
        miss[miss != -999] = 0
        miss[miss == -999] = 1
        col[col == -999] = 0
        miss = miss.reshape(-1, 1)
        if (missing_vectors.shape[0] == 0):
            missing_vectors = miss
        else:
             missing_vectors = np.append(missing_vectors, miss, axis = 1)
print(tX[tX == -999].shape)
missing_vectors = missing_vectors[:, (0, 1, 5)] # only take differents columns of missing_vectors

(1580052,)
(0,)


In [83]:
print(np.dot(missing_vectors.T, missing_vectors))

[[ 38114.  33685.  26123.]
 [ 33685. 177457.  99913.]
 [ 26123.  99913.  99913.]]


In [108]:
degrees = np.arange(10)
N = tX.shape[0]
Weights = []
losses = []
for iter_degree, degree in enumerate(degrees):
    polx = build_poly(tX, degree)
    polx = polx.reshape(N, -1)
    polx = np.append(polx, missing_vectors, axis = 1)
    A = np.dot(polx.T, polx)
    b = np.dot(polx.T, y)
    w = np.linalg.solve(A, b)
    loss = loss_really(w,y,polx)
    print('loss ', iter_degree, ': ', loss)
    Weights.append(w)
    losses.append(loss)
ind = np.argmin(losses)
print(ind)
weights = Weights[ind]

loss  0 :  0.342668
loss  1 :  0.25318
loss  2 :  0.225204
loss  3 :  0.213456
loss  4 :  0.205384
loss  5 :  0.201132
loss  6 :  0.19946
loss  7 :  0.194336
loss  8 :  0.186824
loss  9 :  0.1821
9


## Generate predictions and save ouput in csv format for submission:

In [109]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [110]:
tX_test, missing_test = missing(tX_test)
N = tX_test.shape[0]
polx_test = build_poly(tX_test, degrees[ind])
polx_test = polx_test.reshape(N, -1)
polx_test = np.append(polx_test, missing_test, axis = 1)
tX_test = polx_test

(3588434,)
(0,)


In [111]:
OUTPUT_PATH = '../data/submission1.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)